In [1]:
import healpy
import numpy as np
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery

from multiorder import AbstractMultiorderMapBuilder
from min_max_mean_state import MinMaxMeanState, MinMaxMeanStateMerger
from paths import *

In [2]:
class StateMerger(MinMaxMeanStateMerger):
    def __init__(self, threshold: float = 0.16):
        self.threshold = threshold
    
    def validate_state(self, state: MinMaxMeanState) -> bool:
        norm = max(map(abs, [state.min, state.max]))
        if norm == 0.0:
            return True
        return (state.max - state.min) / norm <= self.threshold
    

def ebv(norder: int):
    n_size = healpy.order2nside(norder)
    n_pix = healpy.order2npix(norder)
    
    indexes = np.arange(n_pix)
    ra, dec = healpy.pix2ang(n_size, indexes, nest=True, lonlat=True)
    coord = SkyCoord(ra=ra, dec=dec, unit='deg')
    
    sfd_query = SFDQuery(INPUT_DIR)
    return sfd_query(coord)
    

class Builder(StateMerger, AbstractMultiorderMapBuilder):
    def __init__(self, max_norder, threshold):
        AbstractMultiorderMapBuilder.__init__(self, max_norder)
        StateMerger.__init__(self, threshold)
        
        self.ebv = ebv(max_norder)
        
    def calculate_state(self, index_max_norder: int) -> MinMaxMeanState:
        value = self.ebv[index_max_norder].item()
        return MinMaxMeanState(min=value, max=value, mean=value)

In [3]:
%%time

tiles = Builder(max_norder=3, threshold=0.16).build()

CPU times: user 17.6 ms, sys: 16.9 ms, total: 34.5 ms
Wall time: 40.4 ms


In [6]:
tiles[3].indexes, tiles[3].values

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
 

In [7]:
from mom_builder import mom_from_array

In [10]:
norder = 11
%time ebv_vals = ebv(norder)
%time mom = mom_from_array(ebv_vals.astype(np.float64), norder, 0.16)

CPU times: user 9.08 s, sys: 1.53 s, total: 10.6 s
Wall time: 11 s
CPU times: user 598 ms, sys: 36.3 ms, total: 634 ms
Wall time: 641 ms
